In [12]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from seleniumbase import SB
import time
import pandas as pd
from selenium.webdriver.support.ui import Select



In [22]:


def wait_for_cloudflare(driver, wait_time=3):
    print(f"⏳ Đợi vượt Cloudflare ({wait_time}s)...")
    time.sleep(wait_time)


def perform_login(driver):
    try:
        print("🔐 Đăng nhập Gmail...")
        login_btn = driver.find_element(By.ID, "ctl00_mainContent_btnloginFeId")
        login_btn.click()
        time.sleep(2)

        gmail_btn = driver.find_element(
            By.XPATH, "//a[contains(@href, 'External/Challenge') and contains(@href, 'scheme=Google')]"
        )
        gmail_btn.click()

        home_btn = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="ctl00_lblNavigation"]/a'))
        )
        home_btn.click()
        time.sleep(1)

        print("✅ Đăng nhập thành công.")
    except Exception as e:
        print(f"❌ Lỗi khi login: {e}")
        raise

#---------------------------------HÀM ĐỂ CÀO----------------------------------------
def crawl_personal_data(driver):
    try:
        button_xpath = '//*[@id="ctl00_mainContent_divMain"]/div[2]/div/table/tbody/tr/td/table/tbody/tr[2]/td[2]/ul/li[3]/a'
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, button_xpath))
        )
        button.click()
        print("✅ Đã click nút.")
        time.sleep(1)

        weeks_to_crawl = list(range(20, 31))
        results = []
        week_ranges = [
            "12/05 - 18/05",
            "19/05 - 25/05",
            "26/05 - 01/06",
            "02/06 - 08/06",
            "09/06 - 15/06",
            "16/06 - 22/06",
            "23/06 - 29/06",
            "30/06 - 06/07",
            "07/07 - 13/07",
            "14/07 - 20/07",
            "21/07 - 27/07"
        ]


    

        for week_val in weeks_to_crawl:
            try:
                select_elem = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID, "ctl00_mainContent_drpSelectWeek"))
                )
                select = Select(select_elem)
                select.select_by_value(str(week_val))
                time.sleep(2)

                table = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="aspnetForm"]/table/tbody/tr[1]/td/div/table'))
                )

                results.append({
                    "week": week_ranges[week_val - 20],
                    "raw_html": table.get_attribute("outerHTML")
                })

                print(f"✅ Done week {week_val}")

            except Exception as e:  # ✅ Đặt đúng trong vòng for
                print(f"❌ Lỗi tuần {week_val}: {e}")

        df = pd.DataFrame(results)
        print(df)
        return df

    except Exception as e:
        print(f"❌ Lỗi khi click nút: {e}")

        
#--------------------------------------------------------------------


def wait_for_url_and_click(driver, target_keyword, button_xpath=None, save_html=False):
    print(f"🕵️ Theo dõi URL để tìm '{target_keyword}'...")
    matched = False
    df = None
    while True:
        try:
            current_url = driver.current_url

            if target_keyword in current_url and not matched:
                print(f"🎯 Đã tìm thấy URL: {current_url}")
                matched = True

                if save_html:
                    with open("matched_page.html", "w", encoding="utf-8") as f:
                        f.write(driver.page_source)
                    print("💾 Đã lưu matched_page.html")

                #tùy biến ở đây
                df=crawl_personal_data(driver)

            driver.title
            time.sleep(1)
        except:
            print("🔴 Trình duyệt đã đóng.")
            break
    return df

def bypass_cloudflare_and_wait_for_url(
    url,
    target_keyword,
    wait_time_after_cloudflare=5,
    headless=False,
    save_html_if_matched=False,
):
    df = None
    with SB(uc=True, headless=headless) as sb:
        driver = sb.driver
        driver.set_window_size(1920, 1080)

        print(f"🌐 Truy cập: {url}")
        driver.get(url)

        wait_for_cloudflare(driver, wait_time_after_cloudflare)

        try:
            perform_login(driver)
        except:
            return

        df=wait_for_url_and_click(
            driver,
            target_keyword=target_keyword,
            save_html=save_html_if_matched
        )
    return df


In [23]:
df=bypass_cloudflare_and_wait_for_url(
    url="https://fap.fpt.edu.vn",
    target_keyword="https://fap.fpt.edu.vn/Student.aspx",
    wait_time_after_cloudflare=5,
    headless=False,
    save_html_if_matched=False
)


🌐 Truy cập: https://fap.fpt.edu.vn
⏳ Đợi vượt Cloudflare (5s)...
🔐 Đăng nhập Gmail...
✅ Đăng nhập thành công.
🕵️ Theo dõi URL để tìm 'https://fap.fpt.edu.vn/Student.aspx'...
🎯 Đã tìm thấy URL: https://fap.fpt.edu.vn/Student.aspx
✅ Đã click nút.
✅ Done week 20
✅ Done week 21
✅ Done week 22
✅ Done week 23
✅ Done week 24
✅ Done week 25
✅ Done week 26
✅ Done week 27
✅ Done week 28
✅ Done week 29
✅ Done week 30
             week                                           raw_html
0   12/05 - 18/05  <table>\n        <thead>\n            <tr>\n\n...
1   19/05 - 25/05  <table>\n        <thead>\n            <tr>\n\n...
2   26/05 - 01/06  <table>\n        <thead>\n            <tr>\n\n...
3   02/06 - 08/06  <table>\n        <thead>\n            <tr>\n\n...
4   09/06 - 15/06  <table>\n        <thead>\n            <tr>\n\n...
5   16/06 - 22/06  <table>\n        <thead>\n            <tr>\n\n...
6   23/06 - 29/06  <table>\n        <thead>\n            <tr>\n\n...
7   30/06 - 06/07  <table>\n        <t

In [24]:
print(df)

             week                                           raw_html
0   12/05 - 18/05  <table>\n        <thead>\n            <tr>\n\n...
1   19/05 - 25/05  <table>\n        <thead>\n            <tr>\n\n...
2   26/05 - 01/06  <table>\n        <thead>\n            <tr>\n\n...
3   02/06 - 08/06  <table>\n        <thead>\n            <tr>\n\n...
4   09/06 - 15/06  <table>\n        <thead>\n            <tr>\n\n...
5   16/06 - 22/06  <table>\n        <thead>\n            <tr>\n\n...
6   23/06 - 29/06  <table>\n        <thead>\n            <tr>\n\n...
7   30/06 - 06/07  <table>\n        <thead>\n            <tr>\n\n...
8   07/07 - 13/07  <table>\n        <thead>\n            <tr>\n\n...
9   14/07 - 20/07  <table>\n        <thead>\n            <tr>\n\n...
10  21/07 - 27/07  <table>\n        <thead>\n            <tr>\n\n...


In [27]:
df.to_csv("FULL_schedule.csv", index=False, encoding="utf-8-sig")